# 📚 Understanding GPT-2 Model Weights & Files

This notebook provides a comprehensive guide to understanding the GPT-2 model files, how they are structured, downloaded, and used in practice.

---

## 📁 Overview of GPT-2 Model Files

When you download GPT-2 weights from OpenAI, you get the following files in the `gpt2/124M/` directory:

| File | Size | Purpose |
|------|------|----------|
| `checkpoint` | 77 bytes | Points to the latest checkpoint file |
| `encoder.json` | ~1 MB | Token-to-ID mapping (vocabulary) |
| `vocab.bpe` | ~446 KB | Byte-Pair Encoding merge rules |
| `hparams.json` | 90 bytes | Model hyperparameters/configuration |
| `model.ckpt.data-00000-of-00001` | ~475 MB | **Actual model weights** |
| `model.ckpt.index` | ~5 KB | Index for weight tensors |
| `model.ckpt.meta` | ~461 KB | TensorFlow graph metadata |

Let's explore each file in detail!

---
## 🔧 Setup

In [12]:
import os
import json
import numpy as np
import tensorflow as tf
import requests
from tqdm import tqdm

# Suppress TensorFlow info messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Path to GPT-2 model files
MODEL_DIR = "gpt2/124M"

# Check if we need to download the model files
if not os.path.exists(MODEL_DIR):
    print("📥 GPT-2 model files not found. Downloading...")
    os.makedirs(MODEL_DIR, exist_ok=True)
    
    base_url = "https://openaipublic.blob.core.windows.net/gpt-2/models/124M"
    filenames = [
        "checkpoint",
        "encoder.json",
        "hparams.json",
        "model.ckpt.data-00000-of-00001",
        "model.ckpt.index",
        "model.ckpt.meta",
        "vocab.bpe"
    ]
    
    for filename in filenames:
        url = f"{base_url}/{filename}"
        filepath = os.path.join(MODEL_DIR, filename)
        
        print(f"  Downloading {filename}...", end=" ")
        response = requests.get(url, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        
        with open(filepath, 'wb') as f:
            if total_size > 1024*1024:  # Show progress for large files
                with tqdm(total=total_size, unit='B', unit_scale=True, desc=filename) as pbar:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                        pbar.update(len(chunk))
            else:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
                print("✅")
    
    print("\n✅ Download complete!")

# Display files in directory
print("\n📂 Files in GPT-2 124M directory:")
print("-" * 50)
for file in sorted(os.listdir(MODEL_DIR)):
    size = os.path.getsize(os.path.join(MODEL_DIR, file))
    if size > 1024*1024:
        size_str = f"{size/(1024*1024):.1f} MB"
    elif size > 1024:
        size_str = f"{size/1024:.1f} KB"
    else:
        size_str = f"{size} bytes"
    print(f"  {file:40} {size_str:>10}")

📥 GPT-2 model files not found. Downloading...

model.ckpt.data-00000-of-00001: 100%|██████████| 498M/498M [00:17<00:00, 29.2MB/s] 



✅ Download complete!

📂 Files in GPT-2 124M directory:
--------------------------------------------------
  checkpoint                                 77 bytes
  encoder.json                              1017.9 KB
  hparams.json                               90 bytes
  model.ckpt.data-00000-of-00001             474.7 MB
  model.ckpt.index                             5.1 KB
  model.ckpt.meta                            460.1 KB
  vocab.bpe                                  445.6 KB


---
## 1️⃣ `hparams.json` - Model Configuration

This file contains the **hyperparameters** that define the model architecture. It's the blueprint that tells us how big the model is.

In [13]:
# Load and display hyperparameters
with open(os.path.join(MODEL_DIR, "hparams.json"), "r") as f:
    hparams = json.load(f)

print("🔧 GPT-2 124M Hyperparameters:")
print("-" * 50)
print(json.dumps(hparams, indent=2))

🔧 GPT-2 124M Hyperparameters:
--------------------------------------------------
{
  "n_vocab": 50257,
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12
}


### Understanding Each Hyperparameter:

| Parameter | Value | Meaning |
|-----------|-------|----------|
| `n_vocab` | 50257 | Size of vocabulary (number of unique tokens) |
| `n_ctx` | 1024 | Maximum context length (tokens the model can "see") |
| `n_embd` | 768 | Embedding dimension (size of each token's vector) |
| `n_head` | 12 | Number of attention heads in multi-head attention |
| `n_layer` | 12 | Number of transformer blocks (depth of the model) |

### GPT-2 Model Sizes Comparison:

| Model | Parameters | n_layer | n_head | n_embd |
|-------|------------|---------|--------|--------|
| 124M (Small) | 124 Million | 12 | 12 | 768 |
| 355M (Medium) | 355 Million | 24 | 16 | 1024 |
| 774M (Large) | 774 Million | 36 | 20 | 1280 |
| 1558M (XL) | 1.5 Billion | 48 | 25 | 1600 |

In [14]:
# Calculate approximate number of parameters
n_vocab = hparams['n_vocab']
n_ctx = hparams['n_ctx']
n_embd = hparams['n_embd']
n_head = hparams['n_head']
n_layer = hparams['n_layer']

# Token embeddings: n_vocab * n_embd
token_emb_params = n_vocab * n_embd

# Position embeddings: n_ctx * n_embd
pos_emb_params = n_ctx * n_embd

# Each transformer block has:
# - Attention: 4 * n_embd * n_embd (Q, K, V, Output projections)
# - MLP: 2 * n_embd * (4 * n_embd) = 8 * n_embd^2
# - Layer norms: 4 * n_embd (2 layer norms with weight and bias)
params_per_block = 4 * n_embd * n_embd + 8 * n_embd * n_embd + 4 * n_embd
transformer_params = n_layer * params_per_block

# Final layer norm
final_ln_params = 2 * n_embd

total_params = token_emb_params + pos_emb_params + transformer_params + final_ln_params

print("📊 Parameter Count Breakdown:")
print("-" * 50)
print(f"Token Embeddings:     {token_emb_params:>15,} params")
print(f"Position Embeddings:  {pos_emb_params:>15,} params")
print(f"Transformer Blocks:   {transformer_params:>15,} params")
print(f"Final Layer Norm:     {final_ln_params:>15,} params")
print("-" * 50)
print(f"Total (approx):       {total_params:>15,} params")
print(f"                      ~{total_params/1e6:.0f} Million parameters")

📊 Parameter Count Breakdown:
--------------------------------------------------
Token Embeddings:          38,597,376 params
Position Embeddings:          786,432 params
Transformer Blocks:        84,971,520 params
Final Layer Norm:               1,536 params
--------------------------------------------------
Total (approx):           124,356,864 params
                      ~124 Million parameters


---
## 2️⃣ `encoder.json` - Vocabulary Mapping

This file contains the **token-to-ID mapping**. It maps each token (word piece) to a unique integer ID that the model uses internally.

GPT-2 uses **Byte-Pair Encoding (BPE)** tokenization, which breaks words into subword units.

In [15]:
# Load the encoder (vocabulary)
with open(os.path.join(MODEL_DIR, "encoder.json"), "r") as f:
    encoder = json.load(f)

print(f"📖 Vocabulary Size: {len(encoder):,} tokens")
print("\n🔤 Sample tokens from vocabulary:")
print("-" * 50)

# Show some interesting tokens
sample_tokens = list(encoder.items())[:20]
for token, idx in sample_tokens:
    # Make whitespace visible
    display_token = repr(token)
    print(f"  Token: {display_token:20} → ID: {idx}")

📖 Vocabulary Size: 50,257 tokens

🔤 Sample tokens from vocabulary:
--------------------------------------------------
  Token: '!'                  → ID: 0
  Token: '"'                  → ID: 1
  Token: '#'                  → ID: 2
  Token: '$'                  → ID: 3
  Token: '%'                  → ID: 4
  Token: '&'                  → ID: 5
  Token: "'"                  → ID: 6
  Token: '('                  → ID: 7
  Token: ')'                  → ID: 8
  Token: '*'                  → ID: 9
  Token: '+'                  → ID: 10
  Token: ','                  → ID: 11
  Token: '-'                  → ID: 12
  Token: '.'                  → ID: 13
  Token: '/'                  → ID: 14
  Token: '0'                  → ID: 15
  Token: '1'                  → ID: 16
  Token: '2'                  → ID: 17
  Token: '3'                  → ID: 18
  Token: '4'                  → ID: 19


In [16]:
# Let's look at some specific tokens
print("\n🔍 Looking up specific tokens:")
print("-" * 50)

words_to_find = ["hello", "Hello", " hello", "world", " the", "the", "Ġthe", "ing", "Ġing"]

for word in words_to_find:
    if word in encoder:
        print(f"  '{word}' → ID: {encoder[word]}")
    else:
        print(f"  '{word}' → NOT FOUND (would be split into subwords)")

print("\n💡 Note: 'Ġ' represents a space before the token (GPT-2's way of encoding spaces)")


🔍 Looking up specific tokens:
--------------------------------------------------
  'hello' → ID: 31373
  'Hello' → ID: 15496
  ' hello' → NOT FOUND (would be split into subwords)
  'world' → ID: 6894
  ' the' → NOT FOUND (would be split into subwords)
  'the' → ID: 1169
  'Ġthe' → ID: 262
  'ing' → ID: 278
  'Ġing' → ID: 5347

💡 Note: 'Ġ' represents a space before the token (GPT-2's way of encoding spaces)


In [17]:
# Create reverse mapping (ID to token)
decoder = {v: k for k, v in encoder.items()}

print("\n🔄 Reverse lookup (ID → Token):")
print("-" * 50)
for idx in [0, 1, 100, 1000, 10000, 50256]:
    token = decoder.get(idx, "NOT FOUND")
    print(f"  ID {idx:>6} → Token: {repr(token)}")

print(f"\n📝 Special token: ID 50256 is '<|endoftext|>' - marks end of text")


🔄 Reverse lookup (ID → Token):
--------------------------------------------------
  ID      0 → Token: '!'
  ID      1 → Token: '"'
  ID    100 → Token: '§'
  ID   1000 → Token: 'ale'
  ID  10000 → Token: 'Ġpocket'
  ID  50256 → Token: '<|endoftext|>'

📝 Special token: ID 50256 is '<|endoftext|>' - marks end of text


---
## 3️⃣ `vocab.bpe` - Byte-Pair Encoding Rules

This file contains the **BPE merge rules**. BPE is a compression algorithm that iteratively merges the most frequent pairs of characters/tokens.

### How BPE Works:
1. Start with individual characters
2. Find the most frequent pair of adjacent tokens
3. Merge them into a new token
4. Repeat until vocabulary size is reached

In [18]:
# Load BPE merge rules
with open(os.path.join(MODEL_DIR, "vocab.bpe"), "r", encoding="utf-8") as f:
    bpe_data = f.read()

# Parse the BPE file
bpe_lines = bpe_data.split('\n')
# First line is version, rest are merge rules
bpe_merges = [tuple(line.split()) for line in bpe_lines[1:] if line.strip()]

print(f"📜 Total BPE merge rules: {len(bpe_merges):,}")
print("\n🔀 First 20 merge rules (most common pairs):")
print("-" * 50)
for i, (a, b) in enumerate(bpe_merges[:20]):
    print(f"  Rule {i+1:>3}: '{a}' + '{b}' → '{a}{b}'")

📜 Total BPE merge rules: 50,000

🔀 First 20 merge rules (most common pairs):
--------------------------------------------------
  Rule   1: 'Ġ' + 't' → 'Ġt'
  Rule   2: 'Ġ' + 'a' → 'Ġa'
  Rule   3: 'h' + 'e' → 'he'
  Rule   4: 'i' + 'n' → 'in'
  Rule   5: 'r' + 'e' → 're'
  Rule   6: 'o' + 'n' → 'on'
  Rule   7: 'Ġt' + 'he' → 'Ġthe'
  Rule   8: 'e' + 'r' → 'er'
  Rule   9: 'Ġ' + 's' → 'Ġs'
  Rule  10: 'a' + 't' → 'at'
  Rule  11: 'Ġ' + 'w' → 'Ġw'
  Rule  12: 'Ġ' + 'o' → 'Ġo'
  Rule  13: 'e' + 'n' → 'en'
  Rule  14: 'Ġ' + 'c' → 'Ġc'
  Rule  15: 'i' + 't' → 'it'
  Rule  16: 'i' + 's' → 'is'
  Rule  17: 'a' + 'n' → 'an'
  Rule  18: 'o' + 'r' → 'or'
  Rule  19: 'e' + 's' → 'es'
  Rule  20: 'Ġ' + 'b' → 'Ġb'


In [19]:
print("\n🔀 Last 10 merge rules (least common pairs):")
print("-" * 50)
for i, (a, b) in enumerate(bpe_merges[-10:]):
    rule_num = len(bpe_merges) - 10 + i + 1
    print(f"  Rule {rule_num:>5}: '{a}' + '{b}' → '{a}{b}'")


🔀 Last 10 merge rules (least common pairs):
--------------------------------------------------
  Rule 49991: 'Comm' + 'ission' → 'Commission'
  Rule 49992: 'Ġ(' + '/' → 'Ġ(/'
  Rule 49993: 'âĢ¦' + '."' → 'âĢ¦."'
  Rule 49994: 'Com' + 'par' → 'Compar'
  Rule 49995: 'Ġampl' + 'ification' → 'Ġamplification'
  Rule 49996: 'om' + 'inated' → 'ominated'
  Rule 49997: 'Ġreg' + 'ress' → 'Ġregress'
  Rule 49998: 'ĠColl' + 'ider' → 'ĠCollider'
  Rule 49999: 'Ġinform' + 'ants' → 'Ġinformants'
  Rule 50000: 'Ġg' + 'azed' → 'Ġgazed'


### Understanding BPE Merge Rules:

The merge rules are ordered by frequency:
- **Early rules** (like `'t' + 'h' → 'th'`) are very common and applied first
- **Later rules** create longer tokens from merged subwords

This is why GPT-2 can handle any text - unknown words are broken into known subword pieces!

---
## 4️⃣ `checkpoint` - TensorFlow Checkpoint Pointer

This small file simply points to the latest checkpoint file.

In [ ]:
# Read checkpoint file
with open(os.path.join(MODEL_DIR, "checkpoint"), "r") as f:
    checkpoint_content = f.read()

print("📍 Checkpoint file content:")
print("-" * 50)
print(checkpoint_content)

print("\n💡 This tells TensorFlow which checkpoint files to load")

---
## 5️⃣ Model Checkpoint Files (The Actual Weights!)

The three `model.ckpt.*` files together contain the **actual neural network weights**:

| File | Purpose |
|------|----------|
| `model.ckpt.data-00000-of-00001` | Binary data containing all weight values |
| `model.ckpt.index` | Index/map to locate tensors in the data file |
| `model.ckpt.meta` | TensorFlow graph structure and metadata |

Let's explore what's inside!

In [ ]:
# Get the checkpoint path
ckpt_path = tf.train.latest_checkpoint(MODEL_DIR)
print(f"📂 Checkpoint path: {ckpt_path}")

# List all variables in the checkpoint
print("\n📦 All tensors in the checkpoint:")
print("=" * 70)

variables = tf.train.list_variables(ckpt_path)
total_params = 0

for name, shape in variables:
    num_params = np.prod(shape)
    total_params += num_params
    print(f"{name:50} {str(shape):20} {num_params:>12,} params")

print("=" * 70)
print(f"{'TOTAL':50} {'':<20} {total_params:>12,} params")
print(f"\n🎯 Actual parameter count: {total_params:,} ({total_params/1e6:.1f}M)")

### Understanding the Weight Names:

The naming convention follows this pattern:

```
model/
├── wte          → Word Token Embeddings (vocab_size × emb_dim)
├── wpe          → Word Position Embeddings (context_length × emb_dim)
├── h0/          → Transformer Block 0
│   ├── ln_1/    → Layer Norm 1 (before attention)
│   ├── attn/    → Multi-Head Attention
│   │   ├── c_attn/  → Combined Q, K, V projection
│   │   └── c_proj/  → Output projection
│   ├── ln_2/    → Layer Norm 2 (before MLP)
│   └── mlp/     → Feed-Forward Network
│       ├── c_fc/    → First linear layer (expand)
│       └── c_proj/  → Second linear layer (project back)
├── h1/ ... h11/ → Transformer Blocks 1-11
└── ln_f/        → Final Layer Norm
```

In [ ]:
# Let's load and examine specific weights
print("🔬 Examining specific weight tensors:")
print("=" * 70)

# Token embeddings
wte = tf.train.load_variable(ckpt_path, "model/wte")
print(f"\n1️⃣ Token Embeddings (wte):")
print(f"   Shape: {wte.shape} = (vocab_size, embedding_dim)")
print(f"   This maps each of {wte.shape[0]:,} tokens to a {wte.shape[1]}-dimensional vector")
print(f"   Sample embedding for token 0: {wte[0, :5]}... (first 5 values)")

# Position embeddings
wpe = tf.train.load_variable(ckpt_path, "model/wpe")
print(f"\n2️⃣ Position Embeddings (wpe):")
print(f"   Shape: {wpe.shape} = (max_positions, embedding_dim)")
print(f"   This encodes position information for up to {wpe.shape[0]} tokens")
print(f"   Sample embedding for position 0: {wpe[0, :5]}... (first 5 values)")

In [ ]:
# Examine attention weights from first transformer block
print("\n3️⃣ Attention Weights (Block 0):")
print("-" * 50)

# c_attn combines Q, K, V projections
c_attn_w = tf.train.load_variable(ckpt_path, "model/h0/attn/c_attn/w")
c_attn_b = tf.train.load_variable(ckpt_path, "model/h0/attn/c_attn/b")
print(f"   c_attn weight: {c_attn_w.shape}")
print(f"   → Combines Q, K, V projections: 768 → 3×768 = 2304")
print(f"   → Each of the 12 heads gets 768/12 = 64 dimensions")

c_proj_w = tf.train.load_variable(ckpt_path, "model/h0/attn/c_proj/w")
print(f"   c_proj weight: {c_proj_w.shape}")
print(f"   → Projects concatenated heads back to embedding dimension")

In [ ]:
# Examine MLP weights
print("\n4️⃣ MLP (Feed-Forward) Weights (Block 0):")
print("-" * 50)

c_fc_w = tf.train.load_variable(ckpt_path, "model/h0/mlp/c_fc/w")
c_proj_w = tf.train.load_variable(ckpt_path, "model/h0/mlp/c_proj/w")

print(f"   c_fc weight: {c_fc_w.shape}")
print(f"   → Expands: 768 → 4×768 = 3072 (hidden dimension)")
print(f"   c_proj weight: {c_proj_w.shape}")
print(f"   → Contracts: 3072 → 768 (back to embedding dimension)")
print(f"\n   💡 The 4× expansion is a common design choice in transformers")

In [ ]:
# Examine Layer Norm parameters
print("\n5️⃣ Layer Normalization Parameters:")
print("-" * 50)

ln1_g = tf.train.load_variable(ckpt_path, "model/h0/ln_1/g")
ln1_b = tf.train.load_variable(ckpt_path, "model/h0/ln_1/b")
print(f"   ln_1 gamma (scale): {ln1_g.shape}")
print(f"   ln_1 beta (shift):  {ln1_b.shape}")

ln_f_g = tf.train.load_variable(ckpt_path, "model/ln_f/g")
ln_f_b = tf.train.load_variable(ckpt_path, "model/ln_f/b")
print(f"   ln_f (final) gamma: {ln_f_g.shape}")
print(f"   ln_f (final) beta:  {ln_f_b.shape}")
print(f"\n   💡 Layer Norm has learnable scale (g) and shift (b) parameters")

---
## 6️⃣ Loading All Weights into a Python Dictionary

Now let's see how to load ALL the weights into a structured Python dictionary that can be used with PyTorch.

In [ ]:
def load_gpt2_params_from_tf_ckpt(ckpt_path, hparams):
    """
    Load GPT-2 parameters from TensorFlow checkpoint into a nested dictionary.
    
    The resulting structure:
    {
        'wte': array,           # Token embeddings
        'wpe': array,           # Position embeddings
        'blocks': [
            {                   # Block 0
                'ln_1': {'g': array, 'b': array},
                'attn': {
                    'c_attn': {'w': array, 'b': array},
                    'c_proj': {'w': array, 'b': array}
                },
                'ln_2': {'g': array, 'b': array},
                'mlp': {
                    'c_fc': {'w': array, 'b': array},
                    'c_proj': {'w': array, 'b': array}
                }
            },
            ...                 # Blocks 1-11
        ],
        'ln_f': {'g': array, 'b': array}  # Final layer norm
    }
    """
    # Initialize with empty blocks
    params = {"blocks": [{} for _ in range(hparams["n_layer"])]}
    
    for name, _ in tf.train.list_variables(ckpt_path):
        # Load the variable
        variable_array = np.squeeze(tf.train.load_variable(ckpt_path, name))
        
        # Parse the variable name (skip 'model/' prefix)
        variable_name_parts = name.split("/")[1:]
        
        # Find target dictionary
        target_dict = params
        if variable_name_parts[0].startswith("h"):
            # This is a transformer block parameter
            layer_number = int(variable_name_parts[0][1:])
            target_dict = params["blocks"][layer_number]
            variable_name_parts = variable_name_parts[1:]
        
        # Navigate/create nested dictionaries
        for key in variable_name_parts[:-1]:
            target_dict = target_dict.setdefault(key, {})
        
        # Set the value
        last_key = variable_name_parts[-1]
        target_dict[last_key] = variable_array
    
    return params

# Load all parameters
print("⏳ Loading all GPT-2 weights...")
params = load_gpt2_params_from_tf_ckpt(ckpt_path, hparams)
print("✅ Weights loaded successfully!")

In [ ]:
# Explore the structure
print("\n📊 Loaded parameters structure:")
print("=" * 50)

def print_structure(d, indent=0):
    """Recursively print dictionary structure with array shapes"""
    for key, value in d.items():
        if isinstance(value, dict):
            print(" " * indent + f"📁 {key}/")
            print_structure(value, indent + 4)
        elif isinstance(value, list):
            print(" " * indent + f"📁 {key}/ [{len(value)} blocks]")
            # Just show first block structure
            print(" " * (indent + 4) + "📁 [0]/  (showing first block)")
            print_structure(value[0], indent + 8)
        elif isinstance(value, np.ndarray):
            print(" " * indent + f"🔢 {key}: shape={value.shape}")

print_structure(params)

In [ ]:
# Verify we can access weights
print("\n✅ Verification - Accessing loaded weights:")
print("-" * 50)
print(f"Token embeddings shape: {params['wte'].shape}")
print(f"Position embeddings shape: {params['wpe'].shape}")
print(f"Number of transformer blocks: {len(params['blocks'])}")
print(f"Block 0 attention c_attn weight shape: {params['blocks'][0]['attn']['c_attn']['w'].shape}")
print(f"Final layer norm gamma shape: {params['ln_f']['g'].shape}")

---
## 7️⃣ How Weights are Downloaded

Let's understand the download process from `07. GPT-2_weights_download.py`:

In [ ]:
# The download URLs
base_url = "https://openaipublic.blob.core.windows.net/gpt-2/models"
model_size = "124M"

filenames = [
    "checkpoint",
    "encoder.json", 
    "hparams.json",
    "model.ckpt.data-00000-of-00001",
    "model.ckpt.index",
    "model.ckpt.meta",
    "vocab.bpe"
]

print("🌐 GPT-2 Model Download URLs:")
print("=" * 80)
for filename in filenames:
    url = f"{base_url}/{model_size}/{filename}"
    print(f"  {url}")

### Download Process:

```python
# 1. Create directory
os.makedirs("gpt2/124M", exist_ok=True)

# 2. Download each file
for filename in filenames:
    url = f"{base_url}/{model_size}/{filename}"
    response = requests.get(url, stream=True)
    
    # Save to disk with progress bar
    with open(f"gpt2/124M/{filename}", "wb") as f:
        for chunk in response.iter_content(1024):
            f.write(chunk)

# 3. Load the TensorFlow checkpoint
ckpt_path = tf.train.latest_checkpoint("gpt2/124M")

# 4. Extract weights into Python dictionary
params = load_gpt2_params_from_tf_ckpt(ckpt_path, hparams)
```

---
## 8️⃣ Using Weights with PyTorch

Now let's see how these TensorFlow weights are converted and used in a PyTorch GPT-2 model.

In [ ]:
import torch
import torch.nn as nn

def assign_weights(pytorch_layer, params_dict, transpose_weights=True):
    """
    Assign numpy weights to PyTorch layer.
    
    Note: TensorFlow uses (input_dim, output_dim) but PyTorch uses (output_dim, input_dim)
    So we need to transpose the weight matrices!
    """
    if 'w' in params_dict:
        weight = params_dict['w']
        if transpose_weights and len(weight.shape) == 2:
            weight = weight.T  # Transpose for PyTorch
        pytorch_layer.weight.data = torch.from_numpy(weight.copy())
    
    if 'b' in params_dict:
        pytorch_layer.bias.data = torch.from_numpy(params_dict['b'].copy())

print("💡 Key insight: TensorFlow vs PyTorch weight shapes")
print("=" * 50)
print("TensorFlow Linear: weight shape = (input_dim, output_dim)")
print("PyTorch Linear:    weight shape = (output_dim, input_dim)")
print("\n→ We must TRANSPOSE weights when converting!")

In [ ]:
# Example: Loading token embeddings into PyTorch
print("\n📥 Loading token embeddings into PyTorch:")
print("-" * 50)

# Create PyTorch embedding layer
token_embedding = nn.Embedding(hparams['n_vocab'], hparams['n_embd'])

# Load weights (no transpose needed for embeddings)
token_embedding.weight.data = torch.from_numpy(params['wte'].copy())

print(f"Original TF shape: {params['wte'].shape}")
print(f"PyTorch layer shape: {token_embedding.weight.shape}")
print(f"\n✅ Embedding loaded successfully!")

# Test it
test_tokens = torch.tensor([50256, 0, 1])  # <|endoftext|> and first two tokens
embeddings = token_embedding(test_tokens)
print(f"\nTest embedding output shape: {embeddings.shape}")
print(f"Embedding for token 0 (first 5 dims): {embeddings[1, :5]}")

---
## 📝 Summary

### What We Learned:

1. **`hparams.json`** - Contains model architecture configuration (layers, heads, dimensions)

2. **`encoder.json`** - Maps tokens to integer IDs (50,257 tokens)

3. **`vocab.bpe`** - Byte-Pair Encoding merge rules for tokenization

4. **`checkpoint`** - Points to the checkpoint files

5. **`model.ckpt.*`** - The actual neural network weights:
   - Token embeddings: (50257, 768)
   - Position embeddings: (1024, 768)
   - 12 Transformer blocks with:
     - Layer norms
     - Multi-head attention (Q, K, V projections)
     - Feed-forward MLP
   - Final layer norm

### Key Takeaways:

- GPT-2 124M has **124 million parameters**
- Weights are stored in **TensorFlow checkpoint format**
- Must **transpose** weights when loading into PyTorch
- The architecture follows the standard transformer decoder pattern

---
## 🔗 Next Steps

Now that you understand the GPT-2 weight files, you can:

1. **Load weights into your own GPT-2 implementation** (from notebook 04)
2. **Generate text** using the pre-trained model
3. **Fine-tune** on your own dataset
4. **Analyze** the learned representations

See the next notebook for loading these weights into the GPT-2 model architecture!